In [170]:
import sys

print(sys.executable)
#!{sys.executable} -m pip install ase
#!{sys.executable} -m pip install tk
import tkinter

print(tkinter.Tcl().eval("info patchlevel"))

/Users/mat3ra/.pyenv/versions/3.11.4/bin/python
8.6.13


In [177]:
# data that comes from JS, here are some examples
poscars = {
    "Ni": """Ni4
1.0
   3.4751458659480110    0.0000000000000000    0.0000000000000002
   0.0000000000000006    3.4751458659480110    0.0000000000000002
   0.0000000000000000    0.0000000000000000    3.4751458659480110
Ni
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Ni
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Ni
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Ni
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Ni
""",
    "Cu": """Cu4
1.0
   3.5774306715697510    0.0000000000000000    0.0000000000000002
   0.0000000000000006    3.5774306715697510    0.0000000000000002
   0.0000000000000000    0.0000000000000000    3.5774306715697510
Cu
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Cu
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Cu
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Cu
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Cu
""",
    "Au": """Au4
1.0
   4.1712885314747270    0.0000000000000000    0.0000000000000003
   0.0000000000000007    4.1712885314747270    0.0000000000000003
   0.0000000000000000    0.0000000000000000    4.1712885314747270
Au
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Au
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Au
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Au
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Au
""",
    "SiC": """Si4 C4
1.0
   4.3539932475828609    0.0000000000000000    0.0000000000000003
   0.0000000000000007    4.3539932475828609    0.0000000000000003
   0.0000000000000000    0.0000000000000000    4.3539932475828609
Si C
4 4
direct
   0.7500000000000000    0.2500000000000000    0.7500000000000000 Si4+
   0.7500000000000000    0.7500000000000000    0.2500000000000000 Si4+
   0.2500000000000000    0.2500000000000000    0.2500000000000000 Si4+
   0.2500000000000000    0.7500000000000000    0.7500000000000000 Si4+
   0.0000000000000000    0.0000000000000000    0.0000000000000000 C4-
   0.0000000000000000    0.5000000000000000    0.5000000000000000 C4-
   0.5000000000000000    0.0000000000000000    0.5000000000000000 C4-
   0.5000000000000000    0.5000000000000000    0.0000000000000000 C4-
""",
    "Graphene": """Graphene
1.0
   2.467291000	   0.000000000	   0.000000000
  -1.233645000	   2.136737000	   0.000000000
   0.000000000	   0.000000000	   7.803074000
C
2
direct
   0.000000000    0.000000000    0.000000000  C
   0.333333000    0.666667000    0.000000000  C
""",
}

In [172]:
# settings
global default_settings
default_settings = {
    "slab": {"miller:h": 1, "miller:k": 1, "miller:l": 1, "vacuum_ratio": 0.5, "number_of_layers": 1},
    "interface": {"slab_v:matrix": [[1, 0], [0, 1]], "layer_v:matrix": [[1, 0], [0, 1]], "distance": 2.0},
}

In [173]:
# definition of MaterialInterface class
from ase import Atoms
from ase.build import surface, supercells
from ase.io import read, write
from ase.visualize import view
import io
from operator import itemgetter
import numpy as np


def poscar_to_atoms(poscar):
    input = io.StringIO(poscar)
    atoms = read(input, format="vasp")
    return atoms


def write_atoms_to_poscar(atoms):
    output = io.StringIO()
    write(output, atoms, format="vasp")
    content = output.getvalue()
    output.close()
    return content


def poscar_to_cif(poscar):
    input = io.StringIO(poscar)
    atoms = read(input, format="vasp")

    output = io.BytesIO()
    write(output, atoms, format="cif")
    content = output.getvalue()
    output.close()
    return content


def expand_matrix_2x2_to_3x3(matrix_2x2):
    matrix_3x3 = [[0, 0, 0], [0, 0, 0], [0, 0, 1]]

    for i in range(2):
        for j in range(2):
            matrix_3x3[i][j] = matrix_2x2[i][j]

    return matrix_3x3


class MaterialInterface:
    def __init__(self, substrate, material, settings=None):
        self.substrate = substrate
        self.material = material
        self.settings = globals()["default_settings"]
        if settings:
            for key in self.settings.keys():
                if key in settings:
                    self.settings[key].update(settings[key])
        self.structure = self.create_structure()

    def create_structure(self):
        slab = self.settings["slab"]
        interface = self.settings["interface"]

        self.substrate = surface(
            self.substrate,
            (slab["miller:h"], slab["miller:k"], slab["miller:l"]),
            vacuum=slab["vacuum_ratio"],
            layers=slab["number_of_layers"],
        )

        interface["slab_v:matrix"] = expand_matrix_2x2_to_3x3(interface["slab_v:matrix"])
        interface["layer_v:matrix"] = expand_matrix_2x2_to_3x3(interface["layer_v:matrix"])

        self.substrate = supercells.make_supercell(self.substrate, interface["slab_v:matrix"])

        self.material = supercells.make_supercell(self.material, interface["layer_v:matrix"])

        z_max_substrate = max(self.substrate.positions[:, 2])
        z_min_material = min(self.material.positions[:, 2])
        z_offset = z_max_substrate - z_min_material + interface["distance"]
        self.material.positions[:, 2] += z_offset

        return self.substrate + self.material

    def calculate_strain(self):
        """Calculates strain for the material layer on the substrate"""

        substrate_cell = self.substrate.get_cell()
        material_cell = self.material.get_cell()

        a0 = np.linalg.norm(substrate_cell[0])
        b0 = np.linalg.norm(substrate_cell[1])

        a1 = np.linalg.norm(material_cell[0])
        b1 = np.linalg.norm(material_cell[1])

        strain_a = (a1 - a0) / a0
        strain_b = (b1 - b0) / b0

        return (strain_a, strain_b)

    def view(self):
        view(self.structure)

In [198]:
# actual Pyodide code
global settings, poscar_data
settings = {
    "slab": {"miller:h": 1, "miller:k": 1, "miller:l": 1, "vacuum_ratio": 0.5, "number_of_layers": 3},
    "interface": {"slab_v:matrix": [[1, 0], [0, 1]], "layer_v:matrix": [[2, 0], [0, 2]], "distance": 3.0},
}
poscar_data = poscars


def func():
    """This function is a gateway to Pyodide in Materials Designer"""

    poscar_data = globals()["poscar_data"]
    settings = globals()["settings"]
    materials = {}

    for material_name, poscar in poscar_data.items():
        materials[material_name] = poscar_to_atoms(poscar)

    interface = MaterialInterface(materials["SiC"], materials["Graphene"], settings)
    write("structure.poscar", interface.structure, format="vasp")

    view(interface.structure)
    print("strain (a, b):", interface.calculate_strain())


func()

strain (a, b): (-0.16350154917538884, -0.16350154130766364)


In [ ]:
interface2 = MaterialInterface(poscar_to_atoms(substrate_poscars[2]), poscar_to_atoms(material_poscar), settings)
print(interface2.structure)
print(interface2.calculate_strain())
interface2.view()

In [ ]:
# interface_master from a paper by Yaoshu Xiea, Kiyou Shibataa and Teruyasu Miozguchia
# https://github.com/nmdl-mizo/interface_master/tree/develop

import sys

# !{sys.executable} -m pip install git+https://github.com/nmdl-mizo/interface_master